In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white, het_breuschpagan, het_goldfeldquandt, linear_reset

# Multicollinearity

Multicollinearity is a statistical phenomenon that occurs when two or more independent variables in a regression model are highly correlated with each other. This high degree of correlation makes it difficult to estimate the unique contribution of each variable to the dependent variable, and it can lead to several problems, such as:

1. Unreliable or unstable coefficients: Multicollinearity can cause the estimated coefficients of the independent variables to become unreliable or unstable. This means that small changes in the data or the model can lead to large changes in the estimated coefficients, making it difficult to interpret the results of the regression analysis.

2. Reduced statistical power: Multicollinearity can reduce the statistical power of the regression analysis, making it more difficult to detect significant relationships between the independent variables and the dependent variable. This can result in a failure to reject the null hypothesis, even when there is a true relationship between the variables.

3. Difficulty in identifying the most important predictors: Multicollinearity can make it difficult to identify the most important predictors of the dependent variable. This is because highly correlated predictors may both appear to be important, even though one of them may be redundant.

4. Inflated standard errors: Multicollinearity can lead to inflated standard errors for the estimated coefficients, which can make it difficult to determine whether the coefficients are statistically significant or not.

Overall, multicollinearity can be a problem because it makes it difficult to estimate the unique contribution of each variable to the dependent variable and can lead to unstable or unreliable results. Therefore, it is important to detect and address multicollinearity when building regression models.

The variance inflation factor (VIF) is a measure of the degree to which the variance of an estimated regression coefficient is increased due to the presence of collinearity in the data. In other words, the VIF measures the extent to which a predictor variable can be linearly predicted from the other predictor variables in the model.

To compute the VIF for a given predictor variable, you need to perform the following steps:

1. Fit a linear regression model with the predictor variable of interest as the dependent variable and all other predictor variables as independent variables.

2. Calculate the R-squared value for this model. The R-squared value represents the proportion of variance in the dependent variable that is explained by the independent variables.

3. Calculate the VIF value for the predictor variable by taking the reciprocal of the tolerance value. The tolerance value is calculated as 1 minus the R-squared value from step 2.

4. Repeat steps 1-3 for each predictor variable in the model.

The VIF ranges from 1 (indicating no collinearity) to infinity (indicating perfect collinearity). In general, a VIF value of 1-2 is considered acceptable, while values greater than 5-10 may indicate significant collinearity.

If high VIF values are detected in a regression analysis, this suggests that the model may suffer from multicollinearity, which can lead to unreliable coefficient estimates and inflated standard errors. To address this issue, you can consider dropping highly correlated variables from the model, combining variables into a single variable, or using regularization techniques such as ridge regression or Lasso regression.

In [ ]:
We use VIF to detect 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import patsy

def VIF(data, formula):  # from explanatory matrix and formula
    """
    Calculates VIF coefficients for given dataframe. Returns DataFrame.
    
    Arguments:
    data - DataFrame that have been used in regression
    formula - Formula used in regression as a string, example: 'Amount ~ C(Transaction_Type) + C(Area)'
    
    Interpretation:
    The variance inflation factor is a measure for the increase of the variance of the parameter estimates 
    if an additional variable, given by exog_idx is added to the linear regression. 
    It is a measure for multicollinearity of the design matrix, exog.

    One recommendation is that if VIF is greater than 5, 
    then the explanatory variable given by exog_idx is highly collinear with the other explanatory variables,
    and the parameter estimates will have large standard errors because of this.
        
    Sources:
    https://en.wikipedia.org/wiki/Variance_inflation_factor
    https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html
    
    """
    f = formula  
    y, X = patsy.dmatrices(f, data, return_type='matrix')

    vif_data = pd.DataFrame()
    vif_data["feature"] = X.design_info.column_names

    vif_values = []
    for i in range(len(X.design_info.column_names)):
        vif_values.append(variance_inflation_factor(X, i))

    vif_data['VIF'] = vif_values
    return vif_data


In [ ]:
def vif_analysis(data):
    """VIF analysis on the data dataframe"""
    for i in range(len(data.columns)):
        v=vif(np.matrix(data),i)
        print("Variance inflation factor for {}: {}".format(data.columns[i],round(v,2)))

# Heteroscedasticity

Heteroscedasticity refers to the situation in which the variance of the residuals (or errors) in a regression model is not constant across all values of the independent variables. This can lead to biased and inefficient Ordinary Least Squares (OLS) estimates.

When heteroscedasticity is present, the standard errors of the OLS estimates will be biased, which means that the estimated standard errors will not reflect the true variability of the estimates. This can lead to incorrect conclusions regarding the statistical significance of the estimated coefficients.

In addition, heteroscedasticity can also lead to inefficient estimates. This means that the estimates will have higher variance and lower precision than they would have if the residuals were homoscedastic (i.e., had constant variance).

One consequence of heteroscedasticity is that it can result in an over-reliance on data points with low error variance, which can lead to less weight being given to data points with high error variance. This can result in an underestimation of the true effect of the independent variable on the dependent variable.

To address the consequences of heteroscedasticity, one approach is to use a weighted least squares (WLS) regression instead of OLS. WLS assigns weights to each data point based on the inverse of the variance of the residuals, thus giving more weight to data points with low error variance and less weight to data points with high error variance.

Another approach is to transform the data in a way that makes the residuals homoscedastic. For example, a common transformation is to take the natural log of the dependent variable, which can help to reduce the impact of heteroscedasticity.

In summary, the consequences of heteroscedasticity for OLS estimates include biased and inefficient estimates, an over-reliance on data points with low error variance, and an underestimation of the true effect of the independent variable on the dependent variable.

Heteroscedasticity is a common problem in regression analysis, and there are several methods to deal with it. Here are some of the most commonly used methods:

1. Weighted Least Squares (WLS): This method involves re-estimating the regression model by assigning weights to each observation based on the inverse of its variance. This gives more weight to observations with smaller variances and less weight to observations with larger variances. WLS can be implemented using specialized software packages, such as STATA or R.

2. Robust Standard Errors: This method involves estimating the standard errors of the regression coefficients using a robust estimator, such as White's estimator or Huber-White estimator. These methods are less sensitive to heteroscedasticity and can produce more accurate standard errors.

3. Data Transformation: One way to deal with heteroscedasticity is to transform the data. Commonly used transformations include taking the logarithm, square root, or inverse of the dependent variable. These transformations can help to stabilize the variance of the residuals and make them more homoscedastic.

4. Heteroscedasticity-Consistent Standard Errors: This method involves estimating the standard errors of the regression coefficients using a formula that accounts for heteroscedasticity. These standard errors are also known as robust standard errors. This method can be used in conjunction with OLS.

5. Generalized Least Squares (GLS): This method involves estimating the regression model by assuming that the residuals follow a specific distribution, such as the normal distribution. GLS can produce more efficient estimates than OLS when the heteroscedasticity is severe, but it requires knowledge of the specific distribution of the residuals.

In summary, there are several methods to deal with heteroscedasticity in regression analysis, including WLS, robust standard errors, data transformation, heteroscedasticity-consistent standard errors, and GLS. The appropriate method depends on the specific characteristics of the data and the research question of interest.

# Endogeneity

Endogeneity refers to a situation in which a predictor variable in a regression model is correlated with the error term or unobserved variables in the model. Endogeneity can arise due to several reasons, such as omitted variables, reverse causality, measurement error, or simultaneity.

Omitted variables refer to variables that are not included in the regression model but are correlated with both the predictor variable and the dependent variable. Omitted variables can lead to biased and inconsistent regression coefficient estimates, which can affect the validity of the conclusions drawn from the analysis.

Reverse causality refers to a situation in which the relationship between the predictor variable and the dependent variable is bidirectional. For example, if the dependent variable is health status and the predictor variable is healthcare expenditure, reverse causality may occur if individuals with poor health status tend to spend more on healthcare.

Measurement error refers to inaccuracies or errors in the measurement of the predictor or dependent variable. Measurement error can lead to biased and inconsistent regression coefficient estimates.

Simultaneity refers to a situation in which the predictor variable and the dependent variable are jointly determined. For example, if the dependent variable is firm profitability and the predictor variable is CEO compensation, simultaneity may occur if CEO compensation is also determined by firm profitability.

Endogeneity can lead to biased and inconsistent regression coefficient estimates, which can affect the validity of the conclusions drawn from the analysis. Therefore, it is important to detect and address endogeneity to ensure that the regression coefficient estimates are unbiased and consistent, and to draw valid conclusions from the analysis.

Endogeneity refers to a situation in which a predictor variable in a regression model is correlated with the error term (or unobserved variables) in the model. Endogeneity can lead to biased and inconsistent regression coefficient estimates, which can affect the validity of the conclusions drawn from the analysis.

Here are some methods to detect endogeneity:

1. Prior knowledge: One way to detect endogeneity is through prior knowledge of the data and the research question. If there are known confounding variables that are likely to affect both the predictor variable and the dependent variable, there may be endogeneity present.

2. Visual inspection: Visual inspection of scatter plots of the predictor variable and the dependent variable can reveal whether there is a linear or nonlinear relationship between the two variables. If there is a strong linear relationship, there may be endogeneity.

3. Correlation analysis: Correlation analysis can reveal whether there is a correlation between the predictor variable and the error term in the model. If there is a significant correlation, this suggests the presence of endogeneity.

4. Instrumental Variables (IV) analysis: IV analysis is a statistical method that can be used to test for endogeneity. It involves identifying an instrumental variable that is correlated with the predictor variable, but is not correlated with the error term. If the instrumental variable is found to be significant, this suggests the presence of endogeneity.

5. Hausman Test: The Hausman test is a statistical test that can be used to determine whether endogeneity is present in a regression model. It involves comparing the coefficients from a fixed-effects regression model to those from a random-effects regression model. If the coefficients differ significantly, this suggests the presence of endogeneity.

In summary, endogeneity can be detected through prior knowledge, visual inspection, correlation analysis, instrumental variables analysis, or the Hausman test. It is important to detect and address endogeneity to ensure that the regression coefficient estimates are unbiased and consistent, and to draw valid conclusions from the analysis.

# MISSPECIFICATION

A model is said to be misspecified when it does not accurately capture the true relationship between the predictor variables and the dependent variable. This can occur due to several reasons, such as:

1. Omitted variables: A misspecified model may omit important predictor variables that are related to the dependent variable. This can lead to biased and inconsistent regression coefficient estimates.

2. Wrong functional form: A misspecified model may use the wrong functional form to represent the relationship between the predictor variables and the dependent variable. For example, using a linear model to represent a nonlinear relationship can lead to biased and inconsistent regression coefficient estimates.

3. Measurement error: A misspecified model may include predictor variables with measurement errors or exclude dependent variables with measurement errors. This can lead to biased and inconsistent regression coefficient estimates.

4. Endogeneity: A misspecified model may include endogenous predictor variables that are correlated with the error term or unobserved variables in the model. This can lead to biased and inconsistent regression coefficient estimates.

5. Outliers: A misspecified model may not account for outliers in the data, which can affect the regression coefficient estimates and the interpretation of the results.

6. Incorrect distributional assumptions: A misspecified model may assume an incorrect distribution for the dependent variable or the error term, which can lead to biased and inconsistent regression coefficient estimates.

In summary, a model can be misspecified due to several reasons, including omitted variables, wrong functional form, measurement error, endogeneity, outliers, and incorrect distributional assumptions. It is important to check for misspecification in regression models to ensure that the results are reliable and the conclusions drawn from the analysis are valid.

# MODEL ASSUMPTIONS

OLS (Ordinary Least Squares) is a widely used linear regression method that estimates the parameters of a linear regression model by minimizing the sum of the squared differences between the predicted and observed values. There are several assumptions that underlie OLS, including assumptions about the nature of explanatory variables and disturbances.

1. Linearity: OLS assumes that the relationship between the dependent variable and the explanatory variables is linear. This means that the coefficients of the explanatory variables are constant across the range of the dependent variable.

2. No perfect multicollinearity: OLS assumes that there is no perfect linear relationship between the explanatory variables. This means that no single explanatory variable can be expressed as a linear combination of the other explanatory variables.

3. Exogeneity: OLS assumes that the explanatory variables are exogenous, meaning that they are not influenced by the disturbance term in the regression equation. This means that the explanatory variables are independent of the error term and are not affected by any other variables that are not included in the model.

4. Homoscedasticity: OLS assumes that the disturbances have constant variance across the range of the dependent variable. This means that the variance of the error term is the same for all observations.

5. Normality: OLS assumes that the disturbances are normally distributed. This means that the errors are distributed around a mean of zero and have a constant variance.

6. Independence: OLS assumes that the disturbances are independent of each other. This means that the value of the error term for one observation does not depend on the value of the error term for any other observation.

In summary, OLS assumptions include linearity, no perfect multicollinearity, exogeneity, homoscedasticity, normality, and independence. Violations of these assumptions can lead to biased and inconsistent regression coefficient estimates and affect the validity of the conclusions drawn from the analysis. Therefore, it is important to check for these assumptions when using OLS and to use appropriate methods to correct for any violations.

Ridge regression is a regularized regression method that is used to address the problem of multicollinearity in linear regression models. Like OLS, Ridge regression makes certain assumptions about the nature of explanatory variables and disturbances. However, Ridge regression relaxes some of the assumptions made by OLS.

1. Linearity: Ridge regression assumes that the relationship between the dependent variable and the explanatory variables is linear. This means that the coefficients of the explanatory variables are constant across the range of the dependent variable.

2. No perfect multicollinearity: Ridge regression assumes that there is no perfect linear relationship between the explanatory variables. This means that no single explanatory variable can be expressed as a linear combination of the other explanatory variables. However, Ridge regression allows for the presence of high correlations among the explanatory variables.

3. Exogeneity: Ridge regression assumes that the explanatory variables are exogenous, meaning that they are not influenced by the disturbance term in the regression equation. This means that the explanatory variables are independent of the error term and are not affected by any other variables that are not included in the model.

4. Homoscedasticity: Ridge regression assumes that the disturbances have constant variance across the range of the dependent variable. This means that the variance of the error term is the same for all observations.

5. Normality: Ridge regression assumes that the disturbances are normally distributed. This means that the errors are distributed around a mean of zero and have a constant variance.

6. Independence: Ridge regression assumes that the disturbances are independent of each other. This means that the value of the error term for one observation does not depend on the value of the error term for any other observation.

In summary, Ridge regression assumptions include linearity, no perfect multicollinearity, exogeneity, homoscedasticity, normality, and independence. Ridge regression is a robust method that can handle the presence of high correlations among the explanatory variables, which makes it useful in situations where OLS may fail.

### Violations of other assumptions


